In [1]:
import pandas as pd
import json
import random

# Carregar o dataset positivo (comida)
df_comida = pd.read_csv("../who_dataset/normalized/normalized_merged_dataset.csv")
num_positivos = len(df_comida)
half = num_positivos // 2

# Carregar os dois arquivos JSONL de frases aleatórias
def carregar_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line)["text"] for line in f]

neg_en  = carregar_jsonl("en.jsonl")
neg_pt  = carregar_jsonl("pt.jsonl")

# Shuffle e cortar
random.shuffle(neg_pt)
random.shuffle(neg_en)

negativos_pt = neg_pt[:half]
negativos_en = neg_en[:(num_positivos - half)]  # Caso ímpar, pega o restante


In [2]:
import re

def clean_text(text):
    # Remove region + país em qualquer posição
    text = re.sub(r'\bregion\s*(pt|usa)\b', '', text, flags=re.IGNORECASE)

    # Lowercase
    text = text.lower()

    # Remove pontuação
    text = re.sub(r'[^\w\s]', '', text)

    # Remover múltiplos espaços
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# Aplicar no dataset positivo
df_comida["meal_description"] = df_comida["meal_description"].apply(clean_text)

# Aplicar no dataset negativo (PT + EN)
negativos_pt = [clean_text(t) for t in negativos_pt]
negativos_en = [clean_text(t) for t in negativos_en]


In [3]:
# Criar dataset balanceado
data = []

for frase in df_comida["meal_description"].tolist():
    data.append({"text": frase, "label": 1})

for frase in negativos_pt + negativos_en:
    data.append({"text": frase, "label": 0})

# Embaralhar final
df_final = pd.DataFrame(data)
df_final = df_final.sample(frac=1).reset_index(drop=True)

# Exportar CSV final
df_final.to_csv("meal_detection_balanced.csv", index=False)
